# Acquiring and analysing building data

## Imports

In [27]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mpmath as mp
import requests
import random

In [19]:
%load-ext autoreload
%autreload 2

UsageError: Line magic function `%load-ext` not found.


## Accessing the data

### Functions for converting to and from lon/lat to XYZ tile format

In [142]:
def get_tile(lat_deg, lon_deg, zoom=15):
    """
    A function to get the relevant tile from lat,lon,zoom)
    """   
    lat_rad = mp.radians(lat_deg)
    n = 2 ** zoom
   
    xtile = n * ((lon_deg + 180) / 360)
    ytile = float(n * (1 - (mp.log(mp.tan(lat_rad) + mp.sec(lat_rad)) / np.pi)) / 2)
    return zoom, round(xtile), round(ytile) # 'tile %d/%d/%d '%

In [3]:
def tile2lon(z,x,y):
    return x / 2**z * 360 - 180

def tile2lat(z,x,y):
    n = mp.pi - 2 * mp.pi * y / 2**z;
    return float((180 / mp.pi) * (mp.atan(0.5 * (mp.exp(n) - mp.exp(-n)))))

def tile_bbox(z,x,y):
    '''
    Returns the lat, lon bounding box of a tile
    '''
    w = tile2lon(z,x,y)
    s = tile2lat(z,x,y) 
    e = tile2lon(z,x+1,y)
    n = tile2lat(z,x,y+1)
    return [w,s,e,n]

## Function for calling the api

In [36]:
def osmbuildings_request(latitude:float, longitude:float):
    """
    returns json response with building data from OSMBuildings
    API for a specific latitude and longitude
    """
    
    base_url = "https://data.osmbuildings.org/0.2/anonymous/tile"
    zoom, xtile, ytile = get_tile(latitude, longitude, 15)
    
    url = f"{base_url}/{zoom}/{xtile}/{ytile}.json"
    print(f"URL: {url}")
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0'}
    response = requests.get(url, headers = headers)
 
    print(f"Status code: {response.status_code}")
    json_response = response.json()

    return json_response

### Coordinates of cities

In [111]:
paris_coords = {'upper_left': [48.813898, 2.264216],
                'lower_right': [48.900502, 2.42172]}

berlin_coords = {'upper_left': [52.475607, 13.313794],
                 'lower_right': [52.55571, 13.471299]}

london_coords = {'upper_left': [51.469149, -0.216408],
                 'lower_right': [51.551072, -0.058904]}

brussels_coords = {'upper_left': [50.808751, 4.288857],
                   'lower_right': [50.891855, 4.446361]}

city_bounds = {'paris': paris_coords,
               'berlin': berlin_coords,
               'london': london_coords,
               'brussels': brussels_coords}
cities = ['paris', 'berlin', 'london', 'brussels']

## Number of XYZ tiles for each

In [98]:
def get_tiles(city_coords):
    ul_tile = get_tile(lat_deg=city_coords['upper_left'][0], lon_deg=city_coords['upper_left'][1], zoom=15)
    lr_tile = get_tile(lat_deg=city_coords['lower_right'][0], lon_deg=city_coords['lower_right'][1], zoom=15)
    city_xtiles = np.abs(ul_tile[1] - lr_tile[1])
    city_ytiles = np.abs(ul_tile[2] - lr_tile[2])
    city_tiles = [city_xtiles, city_ytiles]
    return city_tiles

In [107]:
tiles = {}
tiles['paris'] = get_tiles(paris_coords)
tiles['berlin'] = get_tiles(berlin_coords)
tiles['london'] = get_tiles(london_coords)
tiles['brussels'] = get_tiles(brussels_coords)
for city, tile_counts in tiles.items():
    message = f"The tiles covering {city.title()} are a grid of: {tile_counts}."
    print(message)

The tiles covering Paris are a grid of: [14, 12].
The tiles covering Berlin are a grid of: [14, 12].
The tiles covering London are a grid of: [15, 12].
The tiles covering Brussels are a grid of: [15, 12].


## Getting multiple tiles and checking materials documentation for each city

In [103]:
def get_multiple_jsons(city_coords):
    jsons = []
    for i in range(10):
        random_lat = round(random.uniform(city_coords['upper_left'][0], city_coords['lower_right'][0]), 5)
        random_long = round(random.uniform(city_coords['upper_left'][1], city_coords['lower_right'][1]), 5)
        json_response = osmbuildings_request(random_lat, random_long)
        jsons.append(json_response)
    return jsons

In [104]:
def get_materials(city_jsons):
    materials = []
    total_features = 0
    for json_tile in city_jsons:
        total_features += len(json_tile['features'])
        for n in range(len(json_tile['features'])):
            if json_tile['features'][n]['properties'].get('material') != None:
                materials.append(json_tile['features'][n]['properties'].get('material'))
                proportion = len(materials)/len(json_tile['features'])
    proportion = len(materials)/total_features
    return proportion

In [105]:
ouput_strings =[]
for city in cities:
    city_coords = city_bounds[cities.index(city)]
    city_jsons = get_multiple_jsons(city_coords)
    mat_prop = get_materials(city_jsons)
    ouput_strings.append(f"Proportion of materials in {city}: {mat_prop}")

URL: https://data.osmbuildings.org/0.2/anonymous/tile/15/16602/11270.json
Status code: 200
URL: https://data.osmbuildings.org/0.2/anonymous/tile/15/16594/11273.json
Status code: 200
URL: https://data.osmbuildings.org/0.2/anonymous/tile/15/16597/11277.json
Status code: 200
URL: https://data.osmbuildings.org/0.2/anonymous/tile/15/16603/11278.json
Status code: 200
URL: https://data.osmbuildings.org/0.2/anonymous/tile/15/16590/11277.json
Status code: 200
URL: https://data.osmbuildings.org/0.2/anonymous/tile/15/16599/11270.json
Status code: 200
URL: https://data.osmbuildings.org/0.2/anonymous/tile/15/16594/11272.json
Status code: 200
URL: https://data.osmbuildings.org/0.2/anonymous/tile/15/16596/11269.json
Status code: 200
URL: https://data.osmbuildings.org/0.2/anonymous/tile/15/16596/11271.json
Status code: 200
URL: https://data.osmbuildings.org/0.2/anonymous/tile/15/16599/11275.json
Status code: 200
URL: https://data.osmbuildings.org/0.2/anonymous/tile/15/17602/10741.json
Status code: 200

In [106]:
print(*ouput_strings, sep='\n')

Proportion of materials in paris: 0.009349198640116562
Proportion of materials in berlin: 0.03436619718309859
Proportion of materials in london: 0.04201421068890948
Proportion of materials in brussels: 0.0008201950241501869


### The level of documentation for building material is probably too low for any city (highest 0.05%)

## Getting all tiles covering a city, and the respective json files

In [166]:
def get_all_tiles(city):
    starting_tile = []
    city_coords = city_bounds[city]
    city_tile_grid = tiles[city]
    starting_tile.append(get_tile(city_coords['upper_left'][0], city_coords['upper_left'][1])[1])
    starting_tile.append(get_tile(city_coords['upper_left'][0], city_coords['upper_left'][1])[2])
    x_tiles = []
    y_tiles = []
    for x in range(city_tile_grid[0]-1):
        x_tiles.append(starting_tile[0] + x)
    for y in range(city_tile_grid[1]-1):
        y_tiles.append(starting_tile[1] + y)
    return x_tiles, y_tiles

In [159]:
def get_all_tile_jsons(x_tiles, y_tiles, zoom=15):
    city_jsons = {}
    for x in x_tiles:
        for y in y_tiles:
            tile_lat = tile2lat(zoom, x, y)
            tile_lon = tile2lon(zoom, x, y)
            tile = (x, y)
            json_response = osmbuildings_request(tile_lat, tile_lon)
            city_jsons[tile] = json_response
    return city_jsons

In [174]:
paris_x_tiles, paris_y_tiles = get_all_tiles('paris')
paris_jsons = get_all_tile_jsons(paris_x_tiles, paris_y_tiles)

In [175]:
berlin_x_tiles, berlin_y_tiles = get_all_tiles('berlin')
berlin_jsons = get_all_tile_jsons(berlin_x_tiles, berlin_y_tiles)

In [ ]:
london_x_tiles, london_y_tiles = get_all_tiles('london')
london_jsons = get_all_tile_jsons(london_x_tiles, london_y_tiles)

### Brussels returns an error due to some xyz tiles being empty (either reduce size or change function to include 'try'?)

In [ ]:
# brussels_x_tiles, brussels_y_tiles = get_all_tiles('brussels')
# brussels_jsons = get_all_tile_jsons(brussels_x_tiles, brussels_y_tiles)